In [2]:
%cd /content/drive/My Drive/Colab Notebooks/

/content/drive/My Drive/Colab Notebooks


In [21]:
import datetime
import pandas as pd

In [77]:
import re

date_pattern = re.compile('([0-9]+)/([0-9]+)/([0-9]+)')
time_pattern = re.compile('([0-9]+):([0-9]+)')
mothermilk_pattern_right = re.compile('.*右 ([0-9]+)分')
mothermilk_pattern_left = re.compile('.*左 ([0-9]+)分')
milk_pattern = re.compile('.*ミルク ([0-9]+)ml')
expressedmilk_pattern = re.compile('.*搾母乳 ([0-9]+)ml')
small_pattern = re.compile('.*おしっこ')
big_pattern = re.compile('.*うんち')
sleep_pattern_in = re.compile('.*寝る')
sleep_pattern_out = re.compile('.*起きる')

In [79]:
df = pd.DataFrame(columns=['date', 'time', 'mothermilk', 'milk', 'expressedmilk', 'small', 'big', 'sleep'])

with open('data/raw/202007.txt', 'r', encoding='UTF-8') as f:
    lines = f.readlines()
    for l in lines:
        date_result = date_pattern.match(l)
        if date_result:
            date = datetime.date(int(date_result.group(1)), int(date_result.group(2)), int(date_result.group(3)))
            print(date)

        time_result = time_pattern.match(l)
        if time_result:
            time = datetime.time(int(time_result.group(1)), int(time_result.group(2)))
        
        mothermilk_min = 0
        mothermilk_result_right = mothermilk_pattern_right.match(l)
        if mothermilk_result_right:
            mothermilk_min = mothermilk_min + int(mothermilk_result_right.group(1))
        mothermilk_result_left = mothermilk_pattern_left.match(l)
        if mothermilk_result_left:
            mothermilk_min = mothermilk_min + int(mothermilk_result_left.group(1))
        if mothermilk_min > 0:
            df = df.append({
                'date': date,
                'time': time,
                'mothermilk': mothermilk_min
            }, ignore_index=True)
        
        milk_result = milk_pattern.match(l)
        if milk_result:
            df = df.append({
                'date': date,
                'time': time,
                'milk': int(milk_result.group(1))
            }, ignore_index=True)
        
        expressedmilk_result = expressedmilk_pattern.match(l)
        if expressedmilk_result:
            df = df.append({
                'date': date,
                'time': time,
                'expressedmilk': int(expressedmilk_result.group(1))
            }, ignore_index=True)
        
        small_result = small_pattern.match(l)
        if small_result:
            df = df.append({
                'date': date,
                'time': time,
                'small': 1
            }, ignore_index=True)
        
        big_result = big_pattern.match(l)
        if big_result:
            df = df.append({
                'date': date,
                'time': time,
                'big': 1
            }, ignore_index=True)
        
        sleep_result_in = sleep_pattern_in.match(l)
        if sleep_result_in:
            df = df.append({
                'date': date,
                'time': time,
                'sleep': 1
            }, ignore_index=True)
        sleep_result_out = sleep_pattern_out.match(l)
        if sleep_result_out:
            df = df.append({
                'date': date,
                'time': time,
                'sleep': 0
            }, ignore_index=True)

df

2020-07-02
2020-07-03
2020-07-04
2020-07-05
2020-07-06
2020-07-07
2020-07-08
2020-07-09
2020-07-10
2020-07-11
2020-07-12
2020-07-13
2020-07-14
2020-07-15
2020-07-16
2020-07-17
2020-07-18
2020-07-19
2020-07-20
2020-07-21
2020-07-22
2020-07-23
2020-07-24
2020-07-25
2020-07-26
2020-07-27
2020-07-28
2020-07-29
2020-07-30
2020-07-31


,date,time,mothermilk,milk,expressedmilk,small,big,sleep
0,2020-07-02,20:15:00,NaN,NaN,NaN,1,NaN,NaN
1,2020-07-02,20:15:00,NaN,NaN,NaN,NaN,1,NaN
2,2020-07-03,00:00:00,NaN,10,NaN,NaN,NaN,NaN
3,2020-07-03,03:00:00,NaN,10,NaN,NaN,NaN,NaN
4,2020-07-03,11:00:00,10,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1301,2020-07-31,23:30:00,10,NaN,NaN,NaN,NaN,NaN
1302,2020-07-31,23:40:00,NaN,NaN,NaN,NaN,NaN,1
1303,2020-07-31,23:40:00,170,NaN,NaN,NaN,NaN,NaN
1304,2020-07-31,23:40:00,NaN,NaN,NaN,1,NaN,NaN


In [88]:
df.to_csv('data/parsed/202007.tsv', sep='\t')